In [1]:
import tensorflow as tf
from PIL import Image
import numpy as np
import time

In [2]:
print(tf.__version__)

1.4.0


In [3]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

In [17]:
class TFClassifierGraphLoader():

    def __init__(self):
        #self.classification_graph = self.load_graph('inference_graph/frozen_inference_graph.pb')
        self.classification_graph = self.load_graph('inference_graph/optimized_inference_graph.pb')
        #self.classification_graph = self.load_graph('inference_graph/graph.pb')
        self.input_image = self.classification_graph.get_tensor_by_name('image_tensor:0')        
        
        self.detection_classes = self.classification_graph.get_tensor_by_name('detection_classes:0')
        self.detection_number = self.classification_graph.get_tensor_by_name('num_detections:0')
        self.detection_scores = self.classification_graph.get_tensor_by_name('detection_scores:0')
        self.detection_boxes = self.classification_graph.get_tensor_by_name('detection_boxes:0')
        
        self.sess = tf.Session(graph=self.classification_graph)

    def resolve_traffic_light(self, classification):
        switcher = {
            1: "Green",
            2: "Red",
            3: "Green",
            4: "Green",
            5: "Red",
            6: "Red",
            7: "Yellow",
            8: "Unknown",
            9: "Red",
            10: "Green",
            11: "Green",
            12: "Green",
            13: "Red",
            14: "Red"
        }
 
        return switcher.get(classification, "Unknown")        
        
    def run(self, image):
        time0 = time.time()
                
        classes, detection, scores, boxes = self.sess.run([self.detection_classes, self.detection_number, 
                                                    self.detection_scores, self.detection_boxes], 
                                           feed_dict={self.input_image: image})
                
        time1 = time.time()
        print("Time in milliseconds", (time1 - time0) * 1000)
        
        print(self.resolve_traffic_light(int(np.squeeze(classes)[0])))
        
    def load_graph(self, graph_file):
        """Loads a frozen inference graph"""
        graph = tf.Graph()
        with graph.as_default():
            od_graph_def = tf.GraphDef()
            with tf.gfile.GFile(graph_file, 'rb') as fid:
                serialized_graph = fid.read()
                od_graph_def.ParseFromString(serialized_graph)
                tf.import_graph_def(od_graph_def, name='')
              
        return graph

In [18]:
#image = Image.open('test_images/udacity/image7.jpg')
image = Image.open('test_images/bag/image1.png')

#size = 1280, 720
#image.thumbnail(size)

np_image = load_image_into_numpy_array(image)
np_image_expanded = np.expand_dims(np_image, axis=0)

In [21]:
graph_loader = TFClassifierGraphLoader()

In [22]:
graph_loader.run(np_image_expanded)

InvalidArgumentError: No OpKernel was registered to support Op 'Dequantize' with these attrs.  Registered devices: [CPU,GPU], Registered kernels:
  <no registered kernels>

	 [[Node: SecondStageBoxPredictor/ClassPredictor/weights = Dequantize[T=DT_QUINT8, mode="MIN_FIRST"](SecondStageBoxPredictor/ClassPredictor/weights_quantized_const, SecondStageBoxPredictor/ClassPredictor/weights_quantized_min, SecondStageBoxPredictor/ClassPredictor/weights_quantized_max)]]

Caused by op 'SecondStageBoxPredictor/ClassPredictor/weights', defined at:
  File "C:\ProgramData\Miniconda3\envs\traffic-light-classifier\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\ProgramData\Miniconda3\envs\traffic-light-classifier\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\ProgramData\Miniconda3\envs\traffic-light-classifier\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\ProgramData\Miniconda3\en